In [10]:
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import matplotlib
import matplotlib.pyplot as plt
import torch

In [11]:
#reset the environment
def reset():
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "map_grid_lock_add2TL_copy.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1'])

In [12]:
id = {'virtual' : '270329335' ,'KasemRat': 'cluster_272448137_272555800_272555808_7660045934_7710268409'}
detecters = {'QA' : ['156261350#0'], 'QB' : ['-453962283#3'], 'QC' : ['459551209#0', '459551209#3'], 'QD' : ['27702347#0', '27702347#4', '27702347#6']}

In [13]:
import sumolib
import lxml.etree as ET
def get_edge_id(POI):
#     root = ET.Element('additionals')
#     net = sumolib.net.readNet('map_grid_lock_add2TL_copy_newTLS.net.xml')
#     output = []
#     for key in POI:
#         edgeID = net.getEdge(key)
#         numOfLane = edgeID.getLaneNumber()
#         for lane_index in range(numOfLane):
#             output.append(key+'_'+str(lane_index))
#     print(output)
    if POI == ['156261350#0']:
        output = ['156261350#0_0']
    if POI == ['-453962283#3']:
        output = ['-453962283#3_0']
    if POI == ['459551209#0', '459551209#3']:
        output = ['459551209#0_0', '459551209#0_1', '459551209#0_2', '459551209#0_3', '459551209#3_0', '459551209#3_1', '459551209#3_2', '459551209#3_3']
    if POI == ['27702347#0', '27702347#4', '27702347#6']:
        output = ['27702347#0_0', '27702347#4_0', '27702347#6_0']
    return output    

In [14]:
for key in ['QA', 'QB', 'QC', 'QD']:
    get_edge_id(detecters[key])

In [15]:
def get_state():
    
    Length = []
    for key in ['QA', 'QB', 'QC', 'QD']:
        remain_length = 1-(sum([traci.lanearea.getJamLengthMeters(e) for e in get_edge_id(detecters[key])])/len(get_edge_id(detecters[key])))/traci.lanearea.getLength(get_edge_id(detecters[key])[0])
        Length.append(remain_length*100)
    occupancy = []
    for key in ['QA', 'QB', 'QC', 'QD']:
         occupancy.append(sum([traci.lanearea.getLastStepOccupancy(e) for e in get_edge_id(detecters[key])])/len(get_edge_id(detecters[key])))
    occupancy+= Length
    current_phase = [traci.trafficlight.getPhase(id[key]) for key in ['virtual','KasemRat']]
    for i in current_phase:
        current_phase_binary = [0, 0, 0]
        current_phase_binary[int(i//2)] = 1
        occupancy+= current_phase_binary
    return occupancy

In [31]:
def initial(state, threshold_max, threshold_min, FLAG0, FLAG1): 
    ##FLAG = 1 = Yellow FLAG = 0 = GREEN
    print(state)
    Length = state[4:8]
    occupancy = state[0:4]
    s1 = state[8:11] #virtual phase
    s2 = state[11:14] #Kasem phase
    if ((Length[0] <= threshold_max) or (Length[3] <= threshold_max)):
        if FLAG0 == 1:
            print('set1')
            traci.trafficlight.setPhase(id['virtual'], 1) #YELLOW
            traci.trafficlight.setPhase(id['KasemRat'], 1)
            FLAG0 = 0
            FLAG1 = 1
    elif ((occupancy[0] <= threshold_min) or (occupancy[3] <= threshold_min)):
        if FLAG1 == 1:
            print('set3')
            traci.trafficlight.setPhase(id['virtual'], 5)
            traci.trafficlight.setPhase(id['KasemRat'], 3)
            FLAG0 = 1
            FLAG1 = 0
    if FLAG0 == 0:
        print('set2')
        traci.trafficlight.setPhase(id['virtual'], 2) #GREEN
        traci.trafficlight.setPhase(id['KasemRat'], 2)
        FLAG0 = 0 
        FLAG1 = 1
    if FLAG1 == 0:
        print('set0')
        traci.trafficlight.setPhase(id['virtual'], 4)
        FLAG0 = 1
        FLAG1 = 0
    for i in range(5):
        traci.simulationStep()
    reward = sum(get_state()[4:8])   
    return reward, FLAG0, FLAG1

In [32]:
reward_memory = []
def plot_durations():
    print('show')
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(reward_memory, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Step')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [33]:
traci.close()

KeyError: 'default'

In [34]:
def episode0(threshold_max, threshold_min,volume_virtualJ):
    #episode = 0
    reset()
    FLAG0 = 1
    FLAG2 = 1
    traci.simulationStep()
    for seconds in range(2880):
        state = get_state()
        reward, FLAG0, FLAG2 = initial(state, threshold_max, threshold_min, FLAG0, FLAG2)
        reward_memory.append(reward)
         
    plot_durations()
    traci.close()

In [ ]:
episode0(20, 20, 1000) #PEAM

[0.0, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 1, 0, 0, 1, 0, 0]
set3
set0
[0.0, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[0.0, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[0.0, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[0.0, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[1.5439723320158103, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[3.0879446640316206, 0.0, 0.0, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[4.631916996047432, 0.0, 0.8565163765931204, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[6.175889328063241, 0.0, 0.5980288967562912, 0.0, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[6.175889328063241, 0.0, 2.0525741701057028, 1.202067556196658, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 0, 0, 1]
set0
[4.631916996047432, 0.0, 0.5980288967562912, 3.6062026685899746, 100.0, 100.0, 100.0, 100.0, 0, 0, 1, 1, 0, 0]
set0
[6.175889328063241, 0.0, 0.8